# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
loan_df = pd.read_csv("data/loan_info.csv")
account_df = pd.read_csv('data/account_info.csv')
payment_df = pd.read_csv('data/payment_info.csv')

# Altering the Loans table

In [5]:
pd.set_option("display.max_columns", None)

In [7]:
loan_df

,account_id,owner_client_id,owner_disp_id,num_acct_users,owner_birth_date,statement_freq,card_type,card_issue_date,acct_start_date,loan_id,loan_date,loan_amt,total_paid,loan_duration_mos,total_payments,monthly_payments,loan_status,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,2,2,2,2,1945-02-04,POPLATEK MESICNE,NaN,NaN,1993-02-26,4959,1994-01-05,80952,80952.0,24,24,3373.0,A,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,19,25,25,1,1939-04-23,POPLATEK MESICNE,NaN,NaN,1995-04-07,4961,1996-04-29,30276,20184.0,12,8,2523.0,B,21,Tabor,south Bohemia,103347,87,16,7,1,7,67.0,9104,1.5,2.07,123,2299.0,2354
2,25,31,31,1,1962-02-09,POPLATEK MESICNE,NaN,NaN,1996-07-28,4962,1997-12-08,30276,30276.0,12,12,2523.0,A,68,Frydek - Mistek,north Moravia,228848,15,40,18,2,6,57.2,9893,4.0,4.72,96,5623.0,5887
3,37,45,45,1,1952-08-26,POPLATEK MESICNE,NaN,NaN,1997-08-18,4967,1998-10-14,318480,5308.0,60,1,5308.0,D,20,Strakonice,south Bohemia,70646,94,14,3,1,4,58.4,8547,2.6,3.64,120,1563.0,1542
4,38,46,46,1,1940-01-30,POPLATEK TYDNE,NaN,NaN,1997-08-08,4968,1998-04-19,110736,18456.0,48,8,2307.0,C,19,Prachatice,south Bohemia,51428,50,11,3,1,4,52.7,8402,3.1,3.98,120,999.0,1099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,11327,13923,13615,1,1979-02-22,POPLATEK MESICNE,NaN,NaN,1997-10-15,7294,1998-09-27,39168,4896.0,24,3,1632.0,C,7,Melnik,central Bohemia,94725,38,28,1,3,6,63.4,9920,2.2,2.87,130,4289.0,4846
678,11328,13924,13616,1,1952-09-09,POPLATEK MESICNE,NaN,NaN,1996-11-05,7295,1998-07-18,280440,23370.0,60,5,4674.0,C,54,Brno - mesto,south Moravia,387570,0,0,0,1,1,100.0,9897,1.6,1.96,140,18721.0,18696
679,11349,13955,13647,2,1945-10-30,POPLATEK TYDNE,NaN,NaN,1995-05-26,7304,1995-10-29,419880,265924.0,60,38,6998.0,C,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
680,11359,13968,13660,1,1968-04-13,POPLATEK MESICNE,classic,1995-06-13,1994-10-01,7305,1996-08-06,54024,54024.0,12,12,4502.0,A,61,Trebic,south Moravia,117897,139,28,5,1,6,53.8,8814,4.7,5.74,107,2112.0,2059
